* The "Spline" Condition (Smoothness): The key is that these piecewise polynomials are joined smoothly. At each internal "knot" $x_i$ (where two pieces meet), the spline ensures that:
    * The values are equal: $S_{i-1}(x_i) = S_i(x_i) = y_i$. (The curves connect).
    * The first derivatives are equal: $S'_{i-1}(x_i) = S'_i(x_i)$. (The slope is the same).
    * The second derivatives are equal: $S''_{i-1}(x_i) = S''_i(x_i)$. (The curvature is the same).

* However, the conditions only give you: $2n$ equations (from the interpolation condition) and $n-1$ equations (from the smoothness condition at each internal point). This gives a total of $3n - 1$ equations for $3n$ unknowns. We are one condition short of being able to find a unique solution.

In [14]:
import numpy as np
import pandas as pd
import math
from typing import Tuple

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

# Quadratic Spline

## Algorithm

* Conditions
    * **Piecewise Function:** The interpolant $S(x)$ is constructed from multiple quadratic polynomials $S_i(x)$, one for each interval $[x_i, x_{i+1}]$.
    * **Form:** $S_i(x) = a_i x^2 + b_i x + c_i$.
    * **Interpolation:** The spline must pass through all data points.
    * $S_i(x_i) = y_i$
    * $S_i(x_{i+1}) = y_{i+1}$
    * **Smoothness:** The first derivative (slope) must be continuous at all *internal* knots.
    * $S'_{i-1}(x_i) = S'_i(x_i)$

- Algorithm

This implementation solves for the coefficients by first finding the slope $z_i = S'(x_i)$ at each knot.

**Input:** $n+1$ data points $(x_i, y_i)$ and one boundary condition, $z_0 = S'(x_0)$.

**Output:** A table of coefficients $(a_{i,0}, a_{i,1}, a_{i,2})$ for each piecewise quadratic function $S_i(x) = a_{i,2} \cdot x^2 + a_{i,1} \cdot x + a_{i,0}$.

* Calculate Slopes ($z_i$)

We have $n+1$ unknown slopes $z_0, \dots, z_n$. We are given $z_0$ and can find the rest using the $n$ equations derived from the interpolation and smoothness conditions:

$z_i + z_{i+1} = \frac{2(y_{i+1} - y_i)}{h_i}$
where $h_i = x_{i+1} - x_i$.

1.  **Initialize:** Set $z_0$ to the given boundary condition.
2.  **Iterate:** For $i = 0, \dots, n-1$, recursively calculate:
    $z_{i+1} = \frac{2(y_{i+1} - y_i)}{h_i} - z_i$
3.  **Result:** A list of $n+1$ slopes $[z_0, z_1, \dots, z_n]$.

* Calculate Spline Coefficients

For each interval $i = 0, 1, \dots, n-1$:

1.  **Calculate Degree 2 coefficient ($a_{i,2}$):**
    $a_{i,2} = \frac{z_{i+1} - z_i}{2 h_i}$

2.  **Calculate Degree 1 coefficient ($a_{i,1}$):**
    $a_{i,1} = z_i - 2 a_{i,2} x_i$

3.  **Calculate Degree 0 coefficient ($a_{i,0}$):**
    $a_{i,0} = y_i - a_{i,2} x_i^2 - a_{i,1} x_i$

4.  **Store Coefficients:** Save the triplet $(a_{i,0}, a_{i,1}, a_{i,2})$ for the interval $[x_i, x_{i+1}]$.

* Evaluating the Spline at $x_{val}$

1.  **Find the Interval:** Find the index $i$ such that $x_i \le x_{val} \le x_{i+1}$.
2.  **Select Coefficients:** Retrieve the coefficients $a_{i,0}, a_{i,1}, a_{i,2}$ for segment $i$.
3.  **Calculate:** Compute the value:
    $y_{val} = a_{i,2} \cdot x_{val}^2 + a_{i,1} \cdot x_{val} + a_{i,0}$

In [15]:
def quadratic_spline_interpolation(points, z0_condition):
    """
    Calculates the set of quadratic spline functions S_i(x) for each interval
    in standard polynomial form: S_i(x) = a_2*x^2 + a_1*x + a_0.
    
    A boundary condition z_0 = S'(x_0) must be provided.
    
    Args:
        points (list of tuples): List of (x, y) data points, sorted by x.
        z0_condition (float): The boundary condition for the slope at x_0,
                              (e.g., z_0 = S'(x_0) = 0).

    Returns:
        pd.DataFrame: A DataFrame containing the coefficients for each
                      quadratic segment S_i(x).
    """
    n = len(points) - 1
    if n < 1:
        raise ValueError("At least two points are required for interpolation.")
        
    x = np.array([p[0] for p in points])
    y = np.array([p[1] for p in points])
    h = np.zeros(n)
    z = np.zeros(n + 1)
    
    z_steps_data = []

    # --- 1. Solve for all z_i (slopes at knots) ---
    z[0] = z0_condition
    z_steps_data.append({
        'i': 0,
        'x_i': x[0],
        'y_i': y[0],
        'h_i': 'N/A',
        'z_i = S\'(x_i)': z[0],
        'Calculation': 'Boundary Condition'
    })

    for i in range(n):
        h[i] = x[i+1] - x[i]
        if h[i] <= 0:
            raise ValueError(f"Points are not sorted or x values are not distinct at index {i}.")
            
        # z_{i+1} = 2*(y_{i+1} - y_i)/h_i - z_i
        z[i+1] = (2.0 * (y[i+1] - y[i]) / h[i]) - z[i]
        
        z_steps_data.append({
            'i': i + 1,
            'x_i': x[i+1],
            'y_i': y[i+1],
            'h_i': h[i],
            'z_i = S\'(x_i)': z[i+1],
            'Calculation': f"2*({y[i+1]:.4f}-{y[i]:.4f})/{h[i]:.2f} - {z[i]:.4f}"
        })

    z_steps_df = pd.DataFrame(z_steps_data)
    
    # --- 2. Calculate coefficients for each segment ---
    segments_data = []
    for i in range(n):
        # a_i = (z_{i+1} - z_i) / (2 * h_i)
        a_2 = (z[i+1] - z[i]) / (2.0 * h[i])
        
        # b_i = z_i - 2 * a_i * x_i
        a_1 = z[i] - 2.0 * a_2 * x[i]
        
        # c_i = y_i - a_i * x_i^2 - b_i * x_i
        a_0 = y[i] - a_2 * x[i]**2 - a_1 * x[i]
        
        segments_data.append({
            'i': i,
            'Interval': f"[{x[i]}, {x[i+1]}]",
            'a_0 (Degree 0)': a_0,
            'a_1 (Degree 1)': a_1,
            'a_2 (Degree 2)': a_2
        })
        
    segments_df = pd.DataFrame(segments_data, columns=['i', 'Interval', 'a_0 (Degree 0)', 'a_1 (Degree 1)', 'a_2 (Degree 2)'])
    
    return segments_df



## Results

In [16]:
# 1. Define the data points from the image
points = [
    (1.4, 2.4347052),
    (1.5, 2.5473627),
    (1.6, 2.6495552),
    (1.7, 2.7412610),
    (1.8, 2.8225627),
    (1.9, 2.8936474),
    (2.0, 2.9548000),
    (2.1, 3.0064203),
    (2.2, 3.0489801)
]

# 2. Define the boundary condition (z_0 = S'(x_0))
z0_boundary = 0.0

In [17]:
# 2. Generate the spline segments
segments_table = quadratic_spline_interpolation(points, z0_boundary)

print("--- Linear Spline Segments ---")
segments_table.style

--- Linear Spline Segments ---


,i,Interval,a_0 (Degree 0),a_1 (Degree 1),a_2 (Degree 2)
0,0,"[1.4, 1.5]",24.515575,-31.544100,11.265750
1,1,"[1.5, 1.6]",-28.534925,39.189900,-12.312250
2,2,"[1.6, 1.7]",31.819200,-36.252756,11.263580
3,3,"[1.7, 1.8]",-36.291077,43.876982,-12.303990
4,4,"[1.8, 1.9]",40.128470,-41.033626,11.282290
5,5,"[1.9, 2.0]",-44.915152,48.485976,-12.275500
6,6,"[2.0, 2.1]",49.475928,-45.905104,11.322270
7,7,"[2.1, 2.2]",-54.382174,53.007374,-12.228320


## Further Calculation

In [18]:
def evaluate_quadratic_spline(segments_df, x_val):
    """
    Evaluates the piecewise quadratic spline at a specific x value.
    
    Args:
        segments_df (pd.DataFrame): The DataFrame of spline segments.
        x_val (float): The x-value to interpolate at.

    Returns:
        float: The interpolated y-value.
    """
    
    # 1. Find the correct segment
    segment_found = None
    last_x_ip1 = -np.inf
    
    for _, row in segments_df.iterrows():
        interval_str = row['Interval']
        # Parse interval string "[x_i, x_ip1]"
        x_i, x_ip1 = [float(x) for x in interval_str.strip('[]').split(', ')]
        last_x_ip1 = x_ip1
        
        # Use a small tolerance for the upper bound
        if x_i <= x_val <= (x_ip1 + 1e-14):
            segment_found = row
            break
            
    if segment_found is None:
        first_x_i = float(segments_df.iloc[0]['Interval'].strip('[]').split(', ')[0])
        if x_val < first_x_i or x_val > last_x_ip1:
            raise ValueError(f"x_val = {x_val} is outside the interpolation range [{first_x_i}, {last_x_ip1}]")
        raise ValueError(f"Could not find a segment for x_val = {x_val}")

    
    # 2. Get coefficients from the found segment
    i = segment_found['i']
    a_0 = segment_found['a_0 (Degree 0)']
    a_1 = segment_found['a_1 (Degree 1)']
    a_2 = segment_found['a_2 (Degree 2)']
    
    # 3. Apply the formula S_i(x) = a_2 * x^2 + a_1 * x + a_0
    y_val = a_2 * (x_val**2) + a_1 * x_val + a_0
    
    # --- Print Intermediate Steps ---
    print("\n" + "-"*30)
    print("--- Quadratic Spline Evaluation ---")
    print(f"Target value x = {x_val}")
    print(f"Value is in {segment_found['Interval']} (segment i={i})")
    print(f"Using formula: S_{i}(x) = a_2 * x^2 + a_1 * x + a_0")
    print(f"S_{i}({x_val}) = {a_2:.12f} * {x_val}^2 + {a_1:.12f} * {x_val} + {a_0:.12f}")
    print(f"S_{i}({x_val}) = {y_val:.12f}")
    
    return y_val

In [19]:
# 3. Evaluate the spline at a specific point (e.g., x = 1.75)
x_to_find = 1.75
y_found = evaluate_quadratic_spline(segments_table, x_to_find)


------------------------------
--- Quadratic Spline Evaluation ---
Target value x = 1.75
Value is in [1.7, 1.8] (segment i=3)
Using formula: S_3(x) = a_2 * x^2 + a_1 * x + a_0
S_3(1.75) = -12.303990000000 * 1.75^2 + 43.876982000000 * 1.75 + -36.291077300000
S_3(1.75) = 2.812671825000
